In [6]:
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from utils.dataloader import SceneDataset, scene_collate

In [2]:
train_data_path = "data/eth_ucy/pickles/ynet_processed/zara2_train.pkl"
val_data_path = "data/eth_ucy/pickles/ynet_processed/zara2_test.pkl"
homo_mat_path = "data/eth_ucy/homo_mat/"

homo_mat = {}
for scene in [
    "eth",
    "hotel",
    "students001",
    "students003",
    "uni_examples",
    "zara1",
    "zara2",
    "zara3",
]:
    homo_mat[scene] = torch.Tensor(
        np.loadtxt(homo_mat_path + f"{scene}_H.txt")
    )
seg_mask = True

In [13]:
resize = 1
obs_len = 8
pred_len = 12
total_len = 20
batch_size = 8

df_train = pd.read_pickle(train_data_path)
df_val = pd.read_pickle(val_data_path)

In [7]:
train_dataset = SceneDataset(
    df_train, resize=resize, total_len=total_len
)
train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    collate_fn=scene_collate,
)

Prepare Dataset: 100%|██████████| 7/7 [00:00<00:00, 80.18it/s]


In [15]:
for batch, (trajectory, meta, scene) in enumerate(train_loader):
    for i in range(0, len(trajectory), batch_size):
        observed = trajectory[i : i + batch_size, :obs_len, :]       # (batch_size, obs_len, 2)
        gt_future = trajectory[i : i + batch_size, obs_len:]
        print(observed.shape)
    break

torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([8, 8, 2])
torch.Size([5, 8, 2])
